In [1]:
import openai
import yaml
from src import utils
import time
from pprint import pprint
import os
import argparse
from tqdm import tqdm

In [2]:
# PROMPT = {
#     '3p' : get_3p_prompt,
#     'cot' : get_CoT_prompt,
# }

In [3]:
# api = 'openai'
api_key_file = '/userhomes/philhoon/kt-ai-challenge/api-openai.yaml'

In [ ]:
# model_name="gpt-3.5-turbo-0301",
model_name = "gpt-3.5-turbo-0613"

max_token=2048
temperature=0.0

In [ ]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_preprocess.jsonl'
# output_path = '/userhomes/philhoon/self-debate/result'

In [ ]:
# openai key
key_dict = utils.get_keys(api_key_file)
openai.api_key = key_dict[api]

In [ ]:
output_file = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_reasoning.jsonl'
print(f'output_file : {output_file}')

In [ ]:
# openai-gpt-3.5-turbo-0301-2048-0.0-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-2048-0.0-gsm8k-test.jsonl
# openai-gpt-3.5-turbo-0301-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-inf-1.0-gsm8k-test.jsonl
# openai-gpt-3.5-turbo-0301-2048-0.0-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-2048-0.0-gsm8k-test.jsonl

In [ ]:
# output_file = output_path + '/' + 'gsm8k-korean.jsonl'
# print(output_file)

In [ ]:
# Jsonlines writier
jsonl_writer = utils.JSONLWriter(output_file)

In [ ]:
data = utils.read_jsonlines(input_file)

In [ ]:
# cnt = 0
# for ins in data:
# #     pprint(ins)
#     print(ins['question'])
#     print(ins['answer'])
#     print('----')
#     cnt += 1
#     if cnt == 20:
#         break

In [ ]:
# def get_message(prompt):
#     """
#     Message for chat api
#     """
#     message = {
#         "role" : "user",
#         "content" : prompt
#         }
#     return [message]

In [ ]:
def generate_answer(message, model_name, max_token, temperature):
    """
    Generate response
    """
    try:
        completion = openai.ChatCompletion.create(
            model=model_name,
            messages=message,
            temperature=temperature,
            max_tokens=max_token,
            n=1)
    except:
        print("retrying due to an error......")
        time.sleep(20)
        return generate_answer(message, model_name, max_token, temperature)

    return completion

In [ ]:
# def get_trans_msg(context):
#     """
#     Message for chat api
#     """

#     msg = [
#         {
#             "role" : "system",
#             "content" : "Translate given setences into Korean."
#         },
#         {
#             "role" : "user",
#             "content" : context
#         }
#     ]
    
#     return msg

In [ ]:
def get_reason_mgs(question, answer):
    """
    Message for chat api
    주어진 질문과 답에 대해, 풀이를 수식으로 300자이내로 서술하시오.
    질문: 지름이 12cm, 높이가 14cm인 원기둥의 전개도를 그렸을 때 전개도의 옆면의 둘레는 몇 cm인지 답을 구하시오. (원주율: 3) 답: 100
    """

    msg = [
        {
            "role" : "system",
            "content" : "주어진 질문과 정답에 대해, 풀이를 수식으로 300자이내로 서술하시오."
        },
        {
            "role" : "user",
            "content" : f"질문: {question} 정답:{answer}"
        }
    ]
    
    return msg

In [ ]:
model_name

In [ ]:
max_token

In [ ]:
temperature

In [ ]:
data[0].keys()

In [ ]:
cnt = 0
for instance in tqdm(data):
    query = instance['question']
    ans = instance['answer']
        
    query_msg = get_reason_mgs(query, ans)
    
    
    query_completion = generate_answer(query_msg, model_name, max_token, temperature)
    q_response = query_completion["choices"][0]["message"]["content"]
    
#     print('query_mgs')
#     print(query_msg)
#     print('q_response')
#     print(q_response)
    
    instance['reason'] = q_response
    print(query_msg)
    pprint(instance)

#     jsonl_writer.write_json_line(instance)
    cnt += 1
    if cnt == 10:
        break

In [ ]:
# cnt = 0
# for instance in data:
#     question = instance['question']
#     answer_s = utils.extract_answer(instance['answer'])
#     prompt = get_CoT_prompt(question)
#     prompt = get_prompt(question)
#     print(f'prompt : \n {prompt}')
#     message = get_message(prompt)
    
#     continue
#     completion = generate_answer(message, model_name)
    
# #     completion = openai.ChatCompletion.create(
# #                   model=model_name,
# #                   messages=message,
# #                   temperature=0,Pathways Language Model (PaLM): Scaling to 540 Billion Parameters for Breakthrough Performance
# #                   max_tokens=4096,
# #                   n=1)
    
    
#     response = completion["choices"][0]["message"]["content"]
# #     instance['response'] = response
# #     instance['answer_specific'] = answer_s
# #     jsonl_writer.write_json_line(instance)
#     print(f'respones : \n {response}')
#     cnt += 1
#     if cnt == 10:
#         break

In [ ]:
pprint(instance)